### Writing files to object storage

In [ ]:
import oci

In [ ]:
from pathlib import Path

In [ ]:
config_path = Path.home().joinpath(".oci", "config")

In [ ]:
assert config_path.exists()

In [ ]:
config = oci.config.from_file(config_path, profile_name="DEFAULT")

In [ ]:
config["signer"] = "espoir"

In [ ]:
identity = oci.identity.IdentityClient(config)

In [ ]:
from oci.config import validate_config

In [ ]:
validate_config(config)

In [ ]:
object_storage = oci.object_storage.ObjectStorageClient(config)

In [ ]:
bucket_namespace = "lrtfqsmony6u"
bucket_ocid = "ocid1.bucket.oc1.uk-london-1.aaaaaaaasarm6qfn2ixcr4fauhm7qepd6syacsalqaafq6ucfmjsjd6v2fyq"
bucket_name = "congo_news_qa_dataset"

In [ ]:
def generate_signer_from_config(config):
    # validate the config from file

    # generate the signer using the config info
    signer = oci.signer.Signer(
        tenancy=config["tenancy"],
        user=config["user"],
        fingerprint=config["fingerprint"],
        private_key_file_location=config.get("key_file"),
        # pass_phrase is optional and can be None
        pass_phrase=oci.config.get_config_value_or_default(
            config, "pass_phrase"),
        # private_key_content is optional and can be None
        private_key_content=config.get("key_content")
    )

    return signer

In [ ]:
signer = generate_signer_from_config(config)

In [ ]:
identity_client = oci.identity.IdentityClient(config, signer=signer)

In [ ]:
tenancy_data = identity_client.get_tenancy(config["tenancy"]).data
print(f"My tenancy name is {tenancy_data.name}")

In [ ]:
with open(subset_path, 'rb') as f:
    object_storage.put_object(bucket_namespace, bucket_name, subset_path.name, f)